In [5]:
import numpy as np
import  pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import torch 

import sys

from model import * #Generator,Discriminator
from database import *
from utils import *
# from code.database import ImageDataset

import numpy as np
import gc
import os
import sys

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
from torchvision.transforms.functional import to_pil_image

import itertools
from PIL import Image

import random

from tqdm import tqdm

In [12]:
class TestOpts():
    def __init__(self,seed=1234):
        self.gan_name="cycle_paired"

        self.fix_seed(seed)
        self.experience_ver="cyclegan_paired_ver1.0.0"
        # self.experience_ver="cyclegan_unpaired_ver1.0.0"
        # self.experience_ver="cyclegan_paired_ver1.0.0"
        self.version="0.0.0"
        self.start_epoch = 0
        self.n_epochs = 1
        self.batch_size = 1
        self.dataroot = "../dataset/SUNRGBD/SUNRGBD/kv1/b3dodata/"
        self.lr = 0.0002
        self.decay_epoch = 200
        self.size = 256
        self.domainA_nc = 3
        self.domainB_nc = 3
        self.cpu = False
        self.n_cpu = 0
        self.device_name = "cuda:0" #if torch.cuda.is_available() else "cpu"
        self.device =  torch.device(self.device_name) 
        self.load_weight = True

    def fix_seed(self,seed):
        # Numpy
        np.random.seed(seed)
        random.seed(seed)
        # Pytorch
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
opt=TestOpts()

In [7]:
def disnorm(img):
    IMAGENET_MEAN = [0.5, 0.5, 0.5]
    IMAGENET_STD = [0.5, 0.5, 0.5]

    return 0.5*(img + 1.0)

def create_image(netG_A2B,input_A,input_B,batch):
        # Set model input
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))

        # Generate output
        fake_B = 0.5*(netG_A2B(real_A).data + 1.0)

        # out_img1 = torch.cat([real_A, fake_B,real_B], dim=2)

        A=disnorm(batch["A"])
        B=disnorm(batch["B"])

        

    
        return np.array(to_pil_image(A)),np.array(to_pil_image(B)),np.array(to_pil_image(torch.reshape( fake_B,[3,256,256])))

In [8]:
# import mlflow

""" --- Initial Setting  ---"""

root_path="../output/"
model_path=root_path+f"model/{opt.experience_ver}/"
record_path=root_path+f"generate/{opt.experience_ver}/"
analysis_path=root_path+f"analysis/{opt.experience_ver[:-9]}/"

if not os.path.exists(model_path):
    os.mkdir(model_path)
if not os.path.exists(record_path):
    os.mkdir(record_path)
if not os.path.exists(analysis_path):
    os.mkdir(analysis_path)


# mlflow.set_experiment("depthimage-gan_{}".format(opt.experience_ver))
# mlflow.start_run()
# for _ ,(key , item) in enumerate(vars(opt).items()):
#     mlflow.log_param(key,item)

""" --- Call Models ---"""


# 生成器
netG_A2B = Generator(opt.domainA_nc, opt.domainB_nc)

# GPU
if not opt.cpu:
    netG_A2B.cuda()

# 重みパラメータ初期化
netG_A2B.apply(weights_init_normal)

# 保存したモデルのロード
if opt.load_weight is True:
    netG_A2B.load_state_dict(torch.load(model_path+"netG_A2B.pth", map_location="cuda:0"), strict=False)
   


# 入出力メモリ確保
Tensor = torch.cuda.FloatTensor if not opt.cpu else torch.Tensor
input_A = Tensor(opt.batch_size, opt.domainA_nc, opt.size, opt.size)
input_B = Tensor(opt.batch_size, opt.domainB_nc, opt.size, opt.size)
target_real = Variable(Tensor(opt.batch_size).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(opt.batch_size).fill_(0.0), requires_grad=False)

# 過去データ分のメモリ確保
# データローダー
transforms_ = [ 
                transforms.Resize((int(opt.size),int(opt.size)), Image.BICUBIC), 
                # transforms.RandomCrop(opt.size), 
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
                ]
dataset=ImageDataset(root=opt.dataroot, transforms_=transforms_, unaligned=False,limit=None)
dataloader = DataLoader(dataset, 
                        batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

#Dataset for sampling 
sample_images=dataset

print("num dataloader= {}".format(len(dataloader)))

#Release Memory
# del netD_A,netD_B,fake_A_buffer,fake_B_buffer
# gc.collect()


d:\CODDING\Project\Research\DepthImageGAN\analysis\utils.py:47: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)


num dataloader= 554


In [13]:
results={"color":[],"depth":[],"fake":[]}
folders=os.listdir(opt.dataroot)
for i in tqdm(range(len(dataset))):
    batch=dataset[i]
    _save_path=f"{opt.dataroot}{folders[i]}/{opt.gan_name}/"
    if not os.path.exists(_save_path):
        os.mkdir(_save_path)
    imgs=create_image(netG_A2B,input_A,input_B,batch)
    plt.imsave(f"{_save_path}fake_{i}.png",imgs[2])

    

100%|██████████| 554/554 [01:06<00:00,  8.36it/s]
